<a href="https://colab.research.google.com/github/TomMcIver/Q-Paper/blob/main/Q_Blueprint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y qiskit qiskit-aer qiskit-terra qiskit-ibmq-provider
!pip install qiskit qiskit-aer --upgrade
!pip install PennyLane-Lightning


Found existing installation: qiskit 1.3.2
Uninstalling qiskit-1.3.2:
  Successfully uninstalled qiskit-1.3.2
Found existing installation: qiskit-aer 0.16.1
Uninstalling qiskit-aer-0.16.1:
  Successfully uninstalled qiskit-aer-0.16.1
  Using cached qiskit-1.3.2-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached qiskit_aer-0.16.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.2 kB)
Using cached qiskit-1.3.2-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.8 MB)
Using cached qiskit_aer-0.16.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.4 MB)


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.visualization import plot_bloch_multivector, plot_histogram
import pennylane as qml

plt.rcParams["figure.figsize"] = (8, 6)


In [3]:
import numpy as np

class Qubit:
    def __init__(self):

        self.state = np.array([1, 0], dtype=complex)

    def apply_hadamard(self):
        #apply Hadamard gate
        H = np.array([[1, 1], [1, -1]]) / np.sqrt(2)
        self.state = H @ self.state

    def measure(self):
        # measure the qubit returns 0 or 1 with a rough 50% probability
        return np.random.choice([0, 1], p=np.abs(self.state)**2)


qubit = Qubit()

print("Before Hadamard:", qubit.state)


qubit.apply_hadamard()

print("After Hadamard:", qubit.state)


results = [qubit.measure() for _ in range(10)]
print("Measurement results:", results)


Before Hadamard: [1.+0.j 0.+0.j]
After Hadamard: [0.70710678+0.j 0.70710678+0.j]
Measurement results: [1, 0, 1, 1, 1, 0, 0, 0, 1, 0]


### This code shows the initialization of a Qubit of |0⟩ and |1⟩ in superposition using the Hadamrd gate.  And how the probablityes work when measured.

In [4]:
import numpy as np

class QuantumRegister:
    def __init__(self):

        self.state = np.array([1, 0, 0, 0], dtype=complex)

    def apply_hadamard(self, qubit):
         #hadamard gate to one qubit.
        H = np.array([[1, 1], [1, -1]]) / np.sqrt(2)
        gate = np.kron(H, np.eye(2)) if qubit == 0 else np.kron(np.eye(2), H)
        self.state = gate @ self.state

    def apply_cnot(self):
        """ Apply CNOT gate to entangle qubits. """
        CNOT = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])
        self.state = CNOT @ self.state

    def measure(self):
        """ Measure the qubits and return 00, 01, 10, or 11. """
        return np.random.choice([0, 1, 2, 3], p=np.abs(self.state)**2)

# apply hadamard & cnot aka: entanglement
register = QuantumRegister()
register.apply_hadamard(0)
register.apply_cnot()

# bell state
bell_state = register.state
print(f"Bell State: ({bell_state[0]:.2f})|00⟩ + ({bell_state[3]:.2f})|11⟩")

# measure
results = [f"{register.measure():02b}" for _ in range(10)]
print("Measurement outcomes:", ", ".join(results))


Bell State: (0.71+0.00j)|00⟩ + (0.71+0.00j)|11⟩
Measurement outcomes: 11, 00, 11, 00, 00, 00, 00, 00, 11, 00


### This creates a register of a quantum system that uses two qubits puts one in a superposition and then entangles it with another (more specifically a bell state this is mainly sued for 2 qubits). From there we get the bell state and then measure the system collapsing the qubits into there different states.

In [6]:
import pennylane as qml
import numpy as np

#2 qubits
dev = qml.device("lightning.qubit", wires=2)

# variational quantum circuit
@qml.qnode(dev)
def variational_circuit(params, x):

    qml.RX(x, wires=0)

    # apply parameterized rotations on both qubits
    qml.RY(params[0], wires=0)
    qml.RY(params[1], wires=1)

    # entanglement with a CNOT gate
    qml.CNOT(wires=[0, 1])

    # parameterized rotation on qubit 0
    qml.RZ(params[2], wires=0)

    # mesure expectation values of pauli-z on qubits
    return qml.expval(qml.PauliZ(0)), qml.expval(qml.PauliZ(1))

# compares circuit output to a target output
def cost(params, x, target):
    predictions = variational_circuit(params, x)


    return np.sum((np.array(predictions) - np.array(target)) ** 2)

# initialize parameters randomly with pi
np.random.seed(42)
init_params = np.random.uniform(low=-np.pi, high=np.pi, size=3)

x_input = np.pi / 4
target_output = [0.5, -0.5]

#  optimizer is gradient descent
opt = qml.GradientDescentOptimizer(stepsize=0.1)
steps = 50
params = init_params

# loop to minimize the cost function
for i in range(steps):
    params = opt.step(lambda p: cost(p, x_input, target_output), params)
    if i % 10 == 0:
        current_cost = cost(params, x_input, target_output)
        print(f"Step {i}: Cost = {current_cost:.4f}")

print("Optimized Parameters:", params)
print("Final Circuit Output:", variational_circuit(params, x_input))


Step 0: Cost = 0.0006
Step 10: Cost = 0.0006
Step 20: Cost = 0.0006
Step 30: Cost = 0.0006
Step 40: Cost = 0.0006
Optimized Parameters: [-0.78828768  2.83192151  1.45766093]
Final Circuit Output: (0.4985531554319675, -0.47483889537316776)


This demonstrates a simple parameterized quantum circuit with parameters 0 and 1. These parameters are entangled using a CNOT gate. Gradient descent is then applied as the optimizer to determine the correct parameters.